In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
sess = tf.Session()
optim = tf.train.AdamOptimizer(learning_rate=0.01)

In [4]:
from tqdm import tqdm

In [5]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [6]:
import BayesianNN

In [23]:
bnn = BayesianNN.BayesianFC([1,32,16,1])

In [24]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [25]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [26]:
data_input = list(map(lambda x: [x[0]], data))
data_target = list(map(lambda x: [x[1]], data))

In [27]:
op, data_x, data_y, grads, w_grads, mean_grads, std_grads = bnn.update(1, optim)
sess.run(tf.initialize_all_variables())

In [28]:
for e in tqdm(range(1)):
#     indices = np.random.randint(low=0, high=100, size=batch_size)
#     input_batch = [data_input[i] for i in indices]
#     target_batch = [data_target[i] for i in indices]
    input_batch = data_input
    target_batch = data_target
    
    grad_nums = [a for (a,b) in grads]
    _, wg, mg, sg = sess.run([op, w_grads, mean_grads, std_grads], feed_dict={data_x: input_batch, data_y:target_batch})
    print()
    print(wg)
    print()
    print(mg)
    print()
    print(sg)
    print()
        

#     print("Epoch {}:".format(e))
#     print(sess.run([bnn.qb_mean, bnn.qb_p]))
#     print(g)
#     print()
    #     print("---")
    #     print(sess.run([bnn.qw_p, bnn.qb_p]))    

100%|██████████| 1/1 [00:00<00:00, 10.22it/s]


[array([[  2.71886103e-02,   9.50394679e-06,   0.00000000e+00,
         -2.44141072e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,  -2.69301124e-02,
          8.24058056e-02,   0.00000000e+00,  -1.29216760e-02,
         -3.24673802e-02,   0.00000000e+00,   1.37256876e-01,
          0.00000000e+00,   0.00000000e+00,   1.44647649e-02,
         -1.50000357e-34,   0.00000000e+00,   0.00000000e+00,
         -8.61087292e-02,  -2.73567159e-04,   0.00000000e+00,
         -5.57678193e-02,   0.00000000e+00,   4.07050103e-02,
          0.00000000e+00,  -5.30253798e-02,  -4.37219590e-02,
          0.00000000e+00,  -8.18478263e-36]], dtype=float32), array([[  0.00000000e+00,   3.28382850e-03,   0.00000000e+00,
          0.00000000e+00,   6.33982942e-04,   0.00000000e+00,
         -1.47814304e-03,   0.00000000e+00,   0.00000000e+00,
          5.48148528e-04,   1.70590729e-03,   0.00000000e+00,
         -1.04653537e-02,   0.00000000e+00,   0.00000000e+00,
      

In [18]:
sess.run(bnn.qw_ps[0])

array([[ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]], dtype=float32)

In [13]:
test_set = [[x] for x in np.linspace(-0.5,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [14]:
pred_input, pred_output = bnn.sample()
predictions = sess.run(pred_output, feed_dict={pred_input: test_set})
var_trials = 100
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(pred_output, feed_dict={pred_input: test_set})], axis=1)

100%|██████████| 99/99 [00:00<00:00, 1366.91it/s]


In [15]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[0]

nan

In [16]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [17]:
p = figure(width=500, height=500)
err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")
p.line(data_input_flat, data_target_flat, line_width=3, color="blue")

show(p)